# GPU memory usage for deep learning image processing

### 目的
DLによる画像処理について、モデルの学習と実行に必要なGPUメモリー容量の理論値と実測値の差を検証する。

### 方法
以下の４つの項目について、メモリの使用量の理論値と実測値を比較する。
   
1. 変数
2. Convolutional Neural Network
3. ResNet50
4. Vision Transformer
5. UNETR

メモリの使用量の実測値は、pytorchもしくはtensorflowの関数と、nvml（nvidia  management library）の関数で測定する。
pytorchやtensorflowの関数ではプログラム中で確保された容量が測定される。一方、nvmlではプログラム実行に必要なpythonやcudaなどのオーバーヘッドを含んだ容量が測定される。
変数とUNETRについては２つの実装方法（pytorch, tensorflow）で実測値を測定する。どちらの実装方法でも実測値に大きな差はないことが期待される。

### 結果
|  | 理論値[MiB] | 実測値[MiB] | 誤差[%] |
|---|---|---|---|
| 変数 | 1024 | 1024 | 20 |
| CNN | 636 | 531 |  |
| ResNet50 |  |  |  |
| ViT |  |  |  |
| UNETR |  |  |  |

#### 検証環境
- python 3.10
- tensorflow 2.13.0
- torch 2.0.1
- NVIDIA driver 530.30.02
- CUDA toolkit 11.8
- cuDNN 8.9

#### 参考文献
1. [Estimating GPU Memory Consumption of Deep Learning Models](https://2020.esec-fse.org/details/esecfse-2020-industry-papers/5/Estimating-GPU-Memory-Consumption-of-Deep-Learning-Models), [video](https://dl.acm.org/doi/10.1145/3368089.3417050)
2. [A comprehensive guide to memory usage in PyTorch](https://medium.com/deep-learning-for-protein-design/a-comprehensive-guide-to-memory-usage-in-pytorch-b9b7c78031d3)

## 変数
float32型（4byte）で0.25GiB次元の変数を考える。この変数は理論値で1GiBのサイズである。

変数を定義または削除したときのメモリの使用量を確認して、実測値を求める。
まず、Pytorchでの実測値を求める。

In [1]:
import pynvml
import torch


def print_memory_torch(prefix: str) -> None:
    # Print memory usage

    pynvml.nvmlInit()
    handle = pynvml.nvmlDeviceGetHandleByIndex(0)
    info = pynvml.nvmlDeviceGetMemoryInfo(handle)    
    memory_al = torch.cuda.memory_allocated()
    memory_res = torch.cuda.memory_reserved()
    memory_maxal = torch.cuda.max_memory_allocated()

    print(f"{prefix}: allocated = {memory_al/1024**2:.1f} MiB, "
        f"reserved = {memory_res/1024**2:.1f}MiB, "
        f"max allocated = {memory_maxal/1024**2:.1f} MiB, "
        f"used = {info.used/1024**2:.1f} MiB")


# Define a variable with 1GiB.
dim = 1024**3//4
var_cpu = torch.zeros(dim, dtype=torch.float32)
print(f"var_cpu dtype: {var_cpu.dtype}, dim: {dim/1024**2}MiB")

torch.cuda.reset_peak_memory_stats()
print_memory_torch("Initial")

# Copy the variable to gpu.
var_gpu = var_cpu.to("cuda")
print_memory_torch("Define")

# Delete the variable from gpu.
del var_gpu
print_memory_torch("Delete")

# Release cached memory.
torch.cuda.empty_cache()
print_memory_torch("Release")


var_cpu dtype: torch.float32, dim: 256.0MiB
Initial: allocated = 0.0 MiB, reserved = 0.0MiB, max allocated = 0.0 MiB, used = 602.3 MiB
Define: allocated = 1024.0 MiB, reserved = 1024.0MiB, max allocated = 1024.0 MiB, used = 2426.3 MiB
Delete: allocated = 0.0 MiB, reserved = 1024.0MiB, max allocated = 1024.0 MiB, used = 2426.3 MiB
Release: allocated = 0.0 MiB, reserved = 0.0MiB, max allocated = 1024.0 MiB, used = 1402.3 MiB


変数を定義した後に1024MiB=1GiBのVRAMが確保されたので、実測値は理論値と一致した。
delで変数を削除したあともメモリはreservedとして確保されており、torch.cuda.empty_cache()によって完全にメモリが解放された。

nvmlのusedの結果から、実際には変数のサイズ以上のメモリが確保され、変数を削除した後にもオーバーヘッド分が残っていた。

次にTensorFlowでの実測値を求める。
Tensorflowはデフォルトでプログラム開始時にメモリを最大限確保する。必要なメモリだけを確保するために、memory growthを有効にする。

In [2]:
import tensorflow as tf
import numpy as np


def print_memory_tf(prefix: str) -> None:
    # Print memory usage

    pynvml.nvmlInit()
    handle = pynvml.nvmlDeviceGetHandleByIndex(0)
    info = pynvml.nvmlDeviceGetMemoryInfo(handle)  
    memory_info = tf.config.experimental.get_memory_info("GPU:0")

    print(f"{prefix}: current = {memory_info['current']/1024**2:.1f} MiB, "
        f"peak = {memory_info['peak']/1024**2:.1f}MiB, "
        f"used = {info.used/1024**2:.1f} MiB")


# Enabled memory growth.
devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(devices[0], True)

# Define a variable with 1GiB.
dim = 1024**3//4
var_cpu = np.zeros(dim, dtype=np.float32)
print(f"var_cpu dtype: {var_cpu.dtype}, dim: {dim/1024**2}MiB")

tf.config.experimental.reset_memory_stats("GPU:0")
print_memory_tf("Initail")

# Copy the variable to gpu.
with tf.device("GPU"):
    var_gpu = tf.constant(var_cpu)
print_memory_tf("Define")

# Delete the variable from gpu.
del var_gpu
print_memory_tf("Delete")

2023-08-08 21:50:55.782569: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-08 21:50:55.803908: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-08 21:50:56.212819: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-08 21:50:56.686291: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.

var_cpu dtype: float32, dim: 256.0MiB
Initail: current = 0.0 MiB, peak = 0.0MiB, used = 1588.3 MiB
Define: current = 1024.0 MiB, peak = 1024.0MiB, used = 3634.8 MiB
Delete: current = 0.0 MiB, peak = 1024.0MiB, used = 3634.8 MiB


変数を定義した後に1GiBのVRAMが確保されたので、実測値は理論値と一致した。
オーバーヘッドも含めると、実際には変数のサイズ以上のメモリが確保され、変数を削除したあとも解放されなかった。

## Convolutional Neural Network
画像分類をする単純なCNNについて、メモリー使用量の理論値と実測値を比較する。

データセットは疑似乱数で作成する。
入力画像は3x224x224次元として、1280枚を用意する。
分類クラスは10個として、one-hot encodingで表現する。
バッチサイズは128とする。

CNNは、畳み込み層、平均プーリング層、全結合層を1つずつ持つ構造とする。
畳み込み層は、カーネル3x3、チャンネル数8、ストライド1、パディングなし、バイアスありとする。
平均プーリング層は、カーネル2x2、ストライド2、パディングなしとする。
全結合層は10次元の出力である。
学習の最適化にはSGDを用いる。
数値の精度はPytorchのデフォルトであるfloat32とする。

このCNNに必要なメモリの理論値を求める。
参考文献1,2によれば、学習と推論に必要なメモリは各々以下のようになる。

$$
\begin{align*}
\text{usage for training}   &= \text{data} + \text{weight} + \text{forward output} + \text{weight gradient} + \text{output gradient} \\
                            &= D + W + O * b + W * (d + m) + (O + D) \\
\text{usage for inference}  &= \text{data} + \text{weight} + \text{forward output} \\
                            &= D + W + O \\
\end{align*}
$$

ここで、  
$D$: ミニバッチのデータセットのメモリ使用量  
$W$: 学習パラメーターのメモリ使用量  
$O$: 中間層の出力のメモリ使用量  
$b$: 推論時に中間層の出力のみ半精度にするMixed Precisionを利用する場合は0.5、そうでない場合は1  
$d$: 複数のGPUで実行するDistributed Data Parallelを利用する場合は2、そうでない場合は1  
$m$: 最適化で使うモーメントの数（SGD: 0, Adagrad, RMSprop: 1, Adam: 2）

実際にはこれに加えて、GPUでの計算に必要なメモリー（CUDA context、cuDNN Workspace）とメモリ管理の最適化のための余剰メモリー（Internal Tensor Fragmentation）が使用される。文献1によれば、この追加分はライブラリーのバージョンやモデルによって変わるが、0.5GB程度である。

今回のCNNの場合は、メモリー使用量の理論値は以下のように求まる。

In [3]:
def print_memory_theory1(
    dim_input: list, 
    dim_output: list, 
    num_param: int, 
    num_output_shape: int, 
    moment: int, 
    ddp: int=1, 
    mixed_pre: float=1):
    # Print theoretical memory usage
    
    batchsize
    mem_data = (np.prod(dim_input) + np.prod(dim_output)) * 4
    mem_weight = num_param * 4
    mem_weight_grad = mem_weight * (ddp + moment)
    mem_forward_output = num_output_shape * 4 * mixed_pre
    mem_output_gradient = mem_forward_output + mem_data
    mem_training = mem_data + mem_weight + mem_forward_output + mem_weight_grad + mem_output_gradient
    mem_inference = mem_data + mem_weight + mem_forward_output

    print(f"Data(MiB): {mem_data/1024**2:.1f}")
    print(f"Weight(MiB): {mem_weight/1024**2:.1f}")
    print(f"Forward output(MiB): {mem_forward_output/1024**2:.1f}")
    print(f"Weight gradient(MiB): {mem_weight_grad/1024**2:.1f}")
    print(f"Output gradient(MiB): {mem_output_gradient/1024**2:.1f}")
    print(f"Total for training(MiB): {mem_training/1024**2:.1f}")
    print(f"Total for inference(MiB): {mem_inference/1024**2:.1f}")


batchsize = 128
dim_input = [batchsize, 3, 224, 224]
dim_output = [batchsize, 10]
num_param = 3*3*3*8+8 + 8*((dim_input[2]-2)//2)*((dim_input[3]-2)//2)*10 + 10
num_output_shape = np.prod([batchsize, 8, (dim_input[2]-2), (dim_input[3]-2)]) \
                 + np.prod([batchsize, 8, ((dim_input[2]-2)//2), ((dim_input[3]-2)//2)])
moment = 0 # SGD: 0, Adagrad, RMSprop: 1, Adam: 2
d = 1 # Distributed data parallel: 2, Not: 1
b = 1 # Mixed precision: 0.5, Not: 1

print_memory_theory1(dim_input, dim_output, num_param, num_output_shape, moment, d, b)

Data(MiB): 73.5
Weight(MiB): 3.8
Forward output(MiB): 240.6
Weight gradient(MiB): 3.8
Output gradient(MiB): 314.1
Total for training(MiB): 635.8
Total for inference(MiB): 317.9


よって、理論値は636MiBである。

次にpytorchでCNNを実行したときのメモリ使用量の実測値を調べる。

In [10]:
from torch import nn, optim
import torch.nn.functional as F
from tqdm import tqdm


class CNN(nn.Module):
    def __init__(self, dim_h: int, dim_w: int):
        super().__init__()
        self.conv = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3)
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)
        self.fc = nn.Linear(8*((dim_h-2)//2)*((dim_w-2)//2), 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.conv(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x
    
    def get_device(self):
        return self.fc.weight.device

def train(model:nn.Module, dim_input: list, batchsize: int, epoch: int=3, optimizer: optim.Optimizer=optim.SGD):
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()
    print_memory_torch("Initial")

    model.to("cuda")
    print_memory_torch("Model")

    # dim_input = [1280, 3, 224, 224]
    # model = Net(dim_input[2], dim_input[3])
    # batchsize = dim_input[0]//10
    data = [[torch.randn([batchsize, dim_input[1], dim_input[2], dim_input[3]]), 
             torch.randn(batchsize, 10)] 
             for _ in range(dim_input[0]//batchsize)]

    criterion = F.cross_entropy
    opt = optimizer(model.parameters(), lr=0.01)
    for ep in range(epoch):
        model.train()
        with tqdm(data) as pbar:
            pbar.set_description(f'[Epoch {ep + 1}]')
            for x, y in pbar:
                x = x.to(model.get_device())
                y = y.to(model.get_device())
                
                opt.zero_grad()
                y_pred = model(x)
                loss = criterion(y_pred, y)
                loss.backward()
                opt.step()
            
        print_memory_torch("Train")

    print_memory_torch("Final")

model = CNN(dim_input[2], dim_input[3])
train(model, dim_input=[1280, 3, 224, 224], batchsize=128)

Initial: allocated = 31.9 MiB, reserved = 42.0MiB, max allocated = 31.9 MiB, used = 4491.9 MiB
Model: allocated = 35.7 MiB, reserved = 42.0MiB, max allocated = 35.7 MiB, used = 4493.9 MiB


[Epoch 1]: 100%|██████████| 10/10 [00:00<00:00, 105.86it/s]


Train: allocated = 114.0 MiB, reserved = 604.0MiB, max allocated = 547.2 MiB, used = 5051.7 MiB


[Epoch 2]: 100%|██████████| 10/10 [00:00<00:00, 103.45it/s]


Train: allocated = 114.0 MiB, reserved = 604.0MiB, max allocated = 547.2 MiB, used = 5049.1 MiB


[Epoch 3]: 100%|██████████| 10/10 [00:00<00:00, 107.56it/s]


Train: allocated = 114.0 MiB, reserved = 604.0MiB, max allocated = 547.2 MiB, used = 5049.1 MiB
Final: allocated = 114.0 MiB, reserved = 604.0MiB, max allocated = 547.2 MiB, used = 5049.1 MiB


モデルを用意したときの実測値は3.8MiBであり、理論値と一致した。
学習終了時の実測ピーク値は531MiBであり、理論値と20%の誤差があった。



pytorchでは、torchinfoを使うとネットワーク構造やパラメータ数、メモリー使用量の情報を取得・表示できる。この機能は理論値を計算する際に便利であるが、メモリー使用量の計算は上記理論式と異なる。

具体的には、メモリー使用量＝入力データ＋順伝搬/逆伝搬サイズ＋Weightとしている。順伝搬/逆伝搬サイズForward/backward pass sizeは、学習する層の出力サイズの合計を求め、gradientを考慮してその２倍をメモリー使用量としている。よって、上記理論式とは全く異なる。
また、メモリー使用量の表示がMiBではなくMBである。（1MB = 1000^2byte, 1MiB = 1,024^2byte）

今回のCNNでのtorchinfoの表示は以下のようになる。

In [5]:
from torchinfo import summary

result = summary(model, (batchsize, dim_input[1], dim_input[2], dim_input[3]),
        col_names=("output_size", "num_params", "kernel_size"))
print(result)

# Confirm result
num_input = batchsize * dim_input[1] * dim_input[2] * dim_input[3]
num_param = 3*3*3*8+8 + 8*((dim_input[2]-2)//2)*((dim_input[3]-2)//2)*10 + 10
num_output_shape = np.prod([128, 8, 222, 222]) + np.prod([128, 10])

print("Total prams: ", num_param)
print("Input size(byte): ", num_input * 4)
print("Forward/backward(byte): ", 2 * num_output_shape * 4) # https://github.com/sksq96/pytorch-summary/issues/51
print("Params size(byte): ", num_param * 4)

Layer (type:depth-idx)                   Output Shape              Param #                   Kernel Shape
CNN                                      [128, 10]                 --                        --
├─Conv2d: 1-1                            [128, 8, 222, 222]        224                       [3, 3]
├─AvgPool2d: 1-2                         [128, 8, 111, 111]        --                        2
├─Linear: 1-3                            [128, 10]                 985,690                   --
Total params: 985,914
Trainable params: 985,914
Non-trainable params: 0
Total mult-adds (G): 1.54
Input size (MB): 77.07
Forward/backward pass size (MB): 403.74
Params size (MB): 3.94
Estimated Total Size (MB): 484.76
Total prams:  985914
Input size(byte):  77070336
Forward/backward(byte):  403744768
Params size(byte):  3943656


torchinfoを利用して、定義したmodelから理論値を計算できる。
この方法では、パラメータ数や中間層出力の数を自動的に計算するので簡単である。

In [9]:
def print_memory_theory2(
    model: nn.Module, 
    dim_input: list, 
    moment: int, 
    ddp: int=1, 
    mixed_pre: float=1):
    # Print theoretical memory usage

    info = summary(model, dim_input, verbose=0)
    dim_output = info.summary_list[-1].output_size[1:]

    num_param = 0
    num_output_shape = 0
    last_layer = len(info.summary_list) -1
    for i, layer_info in enumerate(info.summary_list):
        if layer_info.is_leaf_layer:
            num_param += layer_info.trainable_params
            if i != last_layer:
                num_output_shape += np.prod(layer_info.output_size)
    
    mem_data = (np.prod(dim_input) + np.prod(dim_output)) * 4
    mem_weight = num_param * 4
    mem_weight_grad = mem_weight * (ddp + moment)
    mem_forward_output = num_output_shape * 4 * mixed_pre
    mem_output_gradient = mem_forward_output + mem_data
    mem_training = mem_data + mem_weight + mem_forward_output + mem_weight_grad + mem_output_gradient
    mem_inference = mem_data + mem_weight + mem_forward_output

    print(f"Data(MiB): {mem_data/1024**2:.1f}")
    print(f"Weight(MiB): {mem_weight/1024**2:.1f}")
    print(f"Forward output(MiB): {mem_forward_output/1024**2:.1f}")
    print(f"Weight gradient(MiB): {mem_weight_grad/1024**2:.1f}")
    print(f"Output gradient(MiB): {mem_output_gradient/1024**2:.1f}")
    print(f"Total for training(MiB): {mem_training/1024**2:.1f}")
    print(f"Total for inference(MiB): {mem_inference/1024**2:.1f}")


batchsize = 128
dim_input = [batchsize, 3, 224, 224]
moment = 0 # SGD: 0, Adagrad, RMSprop: 1, Adam: 2
d = 1 # Distributed data parallel: 2, Not: 1
b = 1 # Mixed precision: 0.5, Not: 1

print_memory_theory2(model, dim_input, moment, d, b)

Data(MiB): 73.5
Weight(MiB): 3.8
Forward output(MiB): 240.6
Weight gradient(MiB): 3.8
Output gradient(MiB): 314.1
Total for training(MiB): 635.8
Total for inference(MiB): 317.9


## ResNet50